In [3]:
import time
from bs4 import BeautifulSoup
from fake_useragent import UserAgent
import pandas as pd
from urllib.request import Request, urlopen
from urllib.error import URLError, HTTPError

# URL of the site to scrape
base_url = 'https://www.skroutz.gr/c/40/kinhta-thlefwna.html'

# Making a list to store the phones
phones = []

# Loop through the pages, so we can scrape them all
for page_num in range(1, 56):
    # Construct the URL for the current page
    url = f'{base_url}?page={page_num}'
    
    # Generate random agents so the site won't block us.
    headers = {
        "User-Agent": UserAgent().random
    }

    print(f"Processing page {page_num}")
    
    # URL request, if failed we can see which page failed. If we scrape the page successfully we continue.
    try:
        req = Request(url, headers=headers)
        response = urlopen(req)
        page = response.read()
    except HTTPError as e:
        print(f"HTTP error occurred for page {page_num}: {e.code}")
        continue
    except URLError as e:
        print(f"URL error occurred for page {page_num}: {e.reason}")
        continue
    
    # Beautiful Soup to scrape our data
    soup = BeautifulSoup(page, 'html.parser')
    
    # Finding all listings
    listings = soup.find_all('li', class_='cf card')
    
    # Loop through each listing and extract the desired information
    for listing in listings:
        phone = {}  # Making a dictionary to store our info

        # Extract title
        title = listing.find('a', class_="js-sku-link")
        phone['Title'] = title['title'].strip() if title else 'N/A'
        
        # Extract price
        price_tag = listing.find('a', class_='js-sku-link sku-link')
        if price_tag:
            price_text = price_tag.text.replace('από', '').strip()
            phone['Price'] = price_text
        else:
            phone['Price'] = 'N/A'

        # Extract key features
        specs = listing.find('p', class_='specs')
        if specs:
            title_attr = specs.get('title', '')
            details = title_attr.split(', ')
            for detail in details:
                if "Μοντέλο" in detail:
                    phone['Model'] = detail.split(': ')[1]
                elif "Οθόνη" in detail:
                    phone['Screen'] = detail.split(': ')[1]
                elif "Μπαταρία" in detail:
                    phone['Battery'] = detail.split(': ')[1]

        # Extract rating
        rating = listing.find('div', class_='actual-rating')
        phone['Rating'] = rating.text.strip() if rating else 'N/A'

        # Extract number of reviews
        reviews = listing.find('div', class_='reviews-count')
        phone['Reviews'] = reviews.text.strip() if reviews else 'N/A'
        
        # Extract reviews in stars from 1 to 5
        reviews2 = listing.find('a', class_ ="rating stars")
        phone['Stars_Reviews'] = reviews2['title'].strip() if reviews2 else 'N/A'
    
        # Extract shops that have the phone available to order
        stock = listing.find('span', class_ = "shop-count")
        phone['Shop_Stock'] = stock.text.strip() if stock else 'N/A'
        
        # Append the phone details to the phones list
        phones.append(phone)
    # Print results to see if our code works
    print(f"Completed processing page {page_num}")
    print(f"Total phones scraped so far: {len(phones)}")

    # adding delay
    time.sleep(10)

print("Scraping complete")

# Print the extracted data
for phone in phones:
    print(phone)

Processing page 1
Completed processing page 1
Total phones scraped so far: 46
Processing page 2
Completed processing page 2
Total phones scraped so far: 94
Processing page 3
Completed processing page 3
Total phones scraped so far: 142
Processing page 4
Completed processing page 4
Total phones scraped so far: 190
Processing page 5
Completed processing page 5
Total phones scraped so far: 238
Processing page 6
Completed processing page 6
Total phones scraped so far: 286
Processing page 7
Completed processing page 7
Total phones scraped so far: 334
Processing page 8
Completed processing page 8
Total phones scraped so far: 382
Processing page 9
Completed processing page 9
Total phones scraped so far: 430
Processing page 10
Completed processing page 10
Total phones scraped so far: 478
Processing page 11
Completed processing page 11
Total phones scraped so far: 526
Processing page 12
Completed processing page 12
Total phones scraped so far: 574
Processing page 13
Completed processing page 13


In [4]:
# adding our data to a DataFrame

df = pd.DataFrame(phones)
df

,Title,Price,Model,Screen,Battery,Rating,Reviews,Stars_Reviews,Shop_Stock
0,Xiaomi Redmi Note 13 Pro 5G Dual SIM (8GB/256G...,"282,90 €",2024,"OLED 6.67""",5100mAh,134,134,"4,7 αστέρια από 134 χρήστες",σε 52 καταστήματα
1,Samsung Galaxy A55 5G Dual SIM (8GB/128GB) Awe...,"327,94 €",2024,"Super AMOLED 6.6""",5000mAh,71,71,"4,7 αστέρια από 71 χρήστες",σε 70 καταστήματα
2,Apple iPhone 15 5G (6GB/128GB) Μαύρο,"818,46 €",2023,"Super Retina XDR OLED 6.1""",3349mAh,72,72,"4,8 αστέρια από 72 χρήστες",σε 60 καταστήματα
3,Apple iPhone 15 Pro Max 5G (8GB/256GB) Natural...,"1.320,90 €",2023,"Super Retina XDR OLED 6.7""",4441mAh,166,166,"4,7 αστέρια από 166 χρήστες",σε 61 καταστήματα
4,Xiaomi Poco C65 Dual SIM (8GB/256GB) Μαύρο,"126,99 €",2023,"IPS 6.74""",5000mAh,70,70,"4,7 αστέρια από 70 χρήστες",σε 36 καταστήματα
...,...,...,...,...,...,...,...,...,...
2543,Oppo A5 (2020) (3GB/64GB) Mirror Black,"99,99 €",2020,"IPS 6.5""",5000mAh,2,2,"1,0 αστέρια από 2 χρήστες",από Megavison
2544,Doro 6820 Dual SIM Κινητό με Κουμπιά Μαύρο,"90,17 €",2023,"2.8""",NaN,,N/A,Δεν υπάρχει βαθμολογία ακόμα,σε 2 καταστήματα
2545,Doro 780X IUP (512MB/4GB) Dual SIM Κινητό με Κ...,"194,47 €",2021,"2.8""",1600mAh,,N/A,Δεν υπάρχει βαθμολογία ακόμα,από My24shop
2546,Bea-fon C220 Single SIM Κινητό με Κουμπιά Κόκκινο,"35,66 €",2018,"1.77""",800mAh,,N/A,Δεν υπάρχει βαθμολογία ακόμα,από My24shop


In [5]:
df.astype

<bound method NDFrame.astype of                                                   Title       Price Model  \
0     Xiaomi Redmi Note 13 Pro 5G Dual SIM (8GB/256G...    282,90 €  2024   
1     Samsung Galaxy A55 5G Dual SIM (8GB/128GB) Awe...    327,94 €  2024   
2                  Apple iPhone 15 5G (6GB/128GB) Μαύρο    818,46 €  2023   
3     Apple iPhone 15 Pro Max 5G (8GB/256GB) Natural...  1.320,90 €  2023   
4            Xiaomi Poco C65 Dual SIM (8GB/256GB) Μαύρο    126,99 €  2023   
...                                                 ...         ...   ...   
2543             Oppo A5 (2020) (3GB/64GB) Mirror Black     99,99 €  2020   
2544         Doro 6820 Dual SIM Κινητό με Κουμπιά Μαύρο     90,17 €  2023   
2545  Doro 780X IUP (512MB/4GB) Dual SIM Κινητό με Κ...    194,47 €  2021   
2546  Bea-fon C220 Single SIM Κινητό με Κουμπιά Κόκκινο     35,66 €  2018   
2547             Oppo A5 (2020) (3GB/64GB) Mirror Black     99,99 €  2020   

                          Screen  Battery R

In [6]:
df.to_csv('phones_data1.csv', index=False)

In [8]:
# Let's check our data
df

,Title,Price,Model,Screen,Battery,Rating,Reviews,Stars_Reviews,Shop_Stock
0,Xiaomi Redmi Note 13 Pro 5G Dual SIM (8GB/256G...,"282,90 €",2024,"OLED 6.67""",5100mAh,134,134,"4,7 αστέρια από 134 χρήστες",σε 52 καταστήματα
1,Samsung Galaxy A55 5G Dual SIM (8GB/128GB) Awe...,"327,94 €",2024,"Super AMOLED 6.6""",5000mAh,71,71,"4,7 αστέρια από 71 χρήστες",σε 70 καταστήματα
2,Apple iPhone 15 5G (6GB/128GB) Μαύρο,"818,46 €",2023,"Super Retina XDR OLED 6.1""",3349mAh,72,72,"4,8 αστέρια από 72 χρήστες",σε 60 καταστήματα
3,Apple iPhone 15 Pro Max 5G (8GB/256GB) Natural...,"1.320,90 €",2023,"Super Retina XDR OLED 6.7""",4441mAh,166,166,"4,7 αστέρια από 166 χρήστες",σε 61 καταστήματα
4,Xiaomi Poco C65 Dual SIM (8GB/256GB) Μαύρο,"126,99 €",2023,"IPS 6.74""",5000mAh,70,70,"4,7 αστέρια από 70 χρήστες",σε 36 καταστήματα
...,...,...,...,...,...,...,...,...,...
2543,Oppo A5 (2020) (3GB/64GB) Mirror Black,"99,99 €",2020,"IPS 6.5""",5000mAh,2,2,"1,0 αστέρια από 2 χρήστες",από Megavison
2544,Doro 6820 Dual SIM Κινητό με Κουμπιά Μαύρο,"90,17 €",2023,"2.8""",NaN,,N/A,Δεν υπάρχει βαθμολογία ακόμα,σε 2 καταστήματα
2545,Doro 780X IUP (512MB/4GB) Dual SIM Κινητό με Κ...,"194,47 €",2021,"2.8""",1600mAh,,N/A,Δεν υπάρχει βαθμολογία ακόμα,από My24shop
2546,Bea-fon C220 Single SIM Κινητό με Κουμπιά Κόκκινο,"35,66 €",2018,"1.77""",800mAh,,N/A,Δεν υπάρχει βαθμολογία ακόμα,από My24shop
